In [1]:
# !pip install tensorflow
# !pip install matplotlib
# !pip install retrying
# !pip install scikit-learn
# !pip install imblearn

In [2]:
# import global modules
import os
import re
import sys
import time
import json
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
from yaml import safe_load
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from scipy.stats import uniform, randint
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import Recall
from imblearn.over_sampling import RandomOverSampler, SMOTE

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_utils = pth_project / 'utils'
pth_queries = pth_project / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
pth_recommenders = pth_data / 'recommenders'
sys.path.insert(0, pth_project.as_posix())
d_config = safe_load(pth_creds.open())

# import local modules
from utils.gcp import connect_bq_services, connect_pandas_bq_services
from utils.extract import extract_bq_data
from utils.modeling import process_features, extract_stats

2024-02-22 19:48:47.165858: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 19:48:47.166225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 19:48:47.452857: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-22 19:48:48.067435: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-22 19:48:52.822243: W tensorflow/compiler/tf2

In [3]:
bq_client = connect_bq_services(d_config['gcp-project-name'])

In [4]:
%load_ext autoreload
%autoreload 2

#### Extract data

In [5]:
# extract trainning data
sql = f"""
  select *
    from `divg-team-v03-pr-de558a.nba_product_reco_model.nba_training_dataset_v8`
"""
df_train = extract_bq_data(bq_client, sql)
print(df_train.shape)

# extract validation data
sql = f"""
  select *
    from `divg-team-v03-pr-de558a.nba_product_reco_model.nba_test_dataset_v8` 
"""
df_validation = extract_bq_data(bq_client, sql)
print(df_validation.shape)

(462524, 265)
(55505, 265)


#### Process data

In [6]:
d_target_mapping = {
 'sing_acquisition': 0,
 'shs_acquisition': 1,
 'tos_acquisition': 2,
 'wifi_acquisition': 3,
 'ttv_acquisition': 4,
 'sws_acquisition': 5,
 'hsic_acquisition': 6,
 'lwc_acquisition': 7,
 'hpro_acquisition': 8,
 'whsia_acquisition': 9
}

# load features metadata
d_features_metadata = safe_load((pth_utils / 'parameters' / 'acquisition_features_v7.yaml').open())

# process training data
df_train_processed = process_features(df_train, d_features_metadata, 'model_scenario', d_target_mapping)
df_validation_processed = process_features(df_validation, d_features_metadata, 'model_scenario', d_target_mapping)

In [7]:
# df_train_all_num = df_train[df_train.select_dtypes(exclude=['object']).columns]
# df_validation_all_num = df_validation[df_train_all_num.columns]

In [8]:
# df_train_all_num_dropped = df_train_all_num.drop(
#     columns=['cust_src_id', 'ban', 'ban_src_id', 'lpds_id',	'label']
# ).fillna(0)

# df_validation_all_num_dropped = df_validation_all_num.drop(
#     columns=['cust_src_id', 'ban', 'ban_src_id', 'lpds_id',	'label']
# ).fillna(0)

In [9]:
X = df_train_processed.drop(columns=['target'])
#X = df_train_all_num_dropped
y = df_train_processed['target']

X_val = df_validation_processed.drop(columns=['target'])
#X_val = df_validation_all_num_dropped
y_val = df_validation_processed['target']

# split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
# Calculate normalization parameters on the training data
scaler = StandardScaler()
scaler.fit(X_train)

# Apply normalization to the training data
X_train_normalized = scaler.transform(X_train)

# Apply normalization to the validation data using the same parameters
X_test_normalized = scaler.transform(X_test)
X_val_normalized = scaler.transform(X_val)

In [11]:
# Apply random oversampling to balance the dataset
# sampling = RandomOverSampler(random_state=42, sampling_strategy=sampling_strategy)
# sampling = SMOTE(random_state=42)

# X_train_resampled, y_train_resampled = sampling.fit_resample(X_train_normalized.astype('float'), y_train)

#### Model

In [12]:
input_size = len(X_train.columns)
output_size = len(d_target_mapping.keys())
input_size, output_size

(90, 10)

In [13]:
# Create the model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=input_size))
model.add(Dense(200, activation='relu'))
model.add(Dense(output_size, activation='softmax'))


# Compile the model
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3)])

In [14]:
# Train the model
model.fit(X_train_normalized, y_train, epochs=1000, batch_size=4096, validation_data=(X_test_normalized, y_test))

Epoch 1/1000
80/80 [==============================] - 3s 17ms/step - loss: 1.6414 - sparse_top_k_categorical_accuracy: 0.7533 - val_loss: 1.4302 - val_sparse_top_k_categorical_accuracy: 0.8224
Epoch 2/1000
80/80 [==============================] - 1s 11ms/step - loss: 1.3896 - sparse_top_k_categorical_accuracy: 0.8337 - val_loss: 1.3719 - val_sparse_top_k_categorical_accuracy: 0.8374
Epoch 3/1000
80/80 [==============================] - 1s 11ms/step - loss: 1.3476 - sparse_top_k_categorical_accuracy: 0.8431 - val_loss: 1.3432 - val_sparse_top_k_categorical_accuracy: 0.8432
Epoch 4/1000
80/80 [==============================] - 1s 11ms/step - loss: 1.3254 - sparse_top_k_categorical_accuracy: 0.8476 - val_loss: 1.3279 - val_sparse_top_k_categorical_accuracy: 0.8459
Epoch 5/1000
80/80 [==============================] - 1s 11ms/step - loss: 1.3110 - sparse_top_k_categorical_accuracy: 0.8509 - val_loss: 1.3170 - val_sparse_top_k_categorical_accuracy: 0.8484
Epoch 6/1000
80/80 [===============

In [15]:
for n in (1, 2, 3):
    probabilities =  model.predict(X_val_normalized)
    results_ranked = np.argsort(-probabilities, axis=1)
    display(extract_stats(n, results_ranked, y_val, d_target_mapping))

1735/1735 [==============================] - 3s 1ms/step


,n_acquisitions,is_prediction_in_top_1,capture_rate_top_1,product
0,3420,369,10.79,sing_acquisition
1,9983,6094,61.04,shs_acquisition
2,14083,8691,61.71,tos_acquisition
3,4508,1009,22.38,wifi_acquisition
4,15586,8181,52.49,ttv_acquisition
5,844,282,33.41,sws_acquisition
6,5615,3338,59.45,hsic_acquisition
7,658,222,33.74,lwc_acquisition
8,36,0,0.00,hpro_acquisition
9,772,457,59.20,whsia_acquisition


1735/1735 [==============================] - 2s 1ms/step


,n_acquisitions,is_prediction_in_top_2,capture_rate_top_2,product
0,3420,808,23.63,sing_acquisition
1,9983,8704,87.19,shs_acquisition
2,14083,11977,85.05,tos_acquisition
3,4508,1779,39.46,wifi_acquisition
4,15586,11600,74.43,ttv_acquisition
5,844,365,43.25,sws_acquisition
6,5615,4307,76.71,hsic_acquisition
7,658,266,40.43,lwc_acquisition
8,36,0,0.00,hpro_acquisition
9,772,567,73.45,whsia_acquisition


1735/1735 [==============================] - 2s 1ms/step


,n_acquisitions,is_prediction_in_top_3,capture_rate_top_3,product
0,3420,1574,46.02,sing_acquisition
1,9983,9418,94.34,shs_acquisition
2,14083,13191,93.67,tos_acquisition
3,4508,2946,65.35,wifi_acquisition
4,15586,13640,87.51,ttv_acquisition
5,844,423,50.12,sws_acquisition
6,5615,4816,85.77,hsic_acquisition
7,658,365,55.47,lwc_acquisition
8,36,0,0.00,hpro_acquisition
9,772,650,84.20,whsia_acquisition


In [16]:
n= 3
probabilities =  model.predict(X_val_normalized)
results_ranked = np.argsort(-probabilities, axis=1)
display(extract_stats(n, results_ranked, y_val, d_target_mapping))

1735/1735 [==============================] - 2s 1ms/step


,n_acquisitions,is_prediction_in_top_3,capture_rate_top_3,product
0,3420,1574,46.02,sing_acquisition
1,9983,9418,94.34,shs_acquisition
2,14083,13191,93.67,tos_acquisition
3,4508,2946,65.35,wifi_acquisition
4,15586,13640,87.51,ttv_acquisition
5,844,423,50.12,sws_acquisition
6,5615,4816,85.77,hsic_acquisition
7,658,365,55.47,lwc_acquisition
8,36,0,0.00,hpro_acquisition
9,772,650,84.20,whsia_acquisition


In [17]:
n= 3
probabilities =  model.predict(X_val_normalized)
results_ranked = np.argsort(-probabilities, axis=1)
display(extract_stats(n, results_ranked, y_val, d_target_mapping))

1735/1735 [==============================] - 2s 1ms/step


,n_acquisitions,is_prediction_in_top_3,capture_rate_top_3,product
0,3420,1574,46.02,sing_acquisition
1,9983,9418,94.34,shs_acquisition
2,14083,13191,93.67,tos_acquisition
3,4508,2946,65.35,wifi_acquisition
4,15586,13640,87.51,ttv_acquisition
5,844,423,50.12,sws_acquisition
6,5615,4816,85.77,hsic_acquisition
7,658,365,55.47,lwc_acquisition
8,36,0,0.00,hpro_acquisition
9,772,650,84.20,whsia_acquisition


In [18]:
n= 3
probabilities =  model.predict(X_val_normalized)
results_ranked = np.argsort(-probabilities, axis=1)
display(extract_stats(n, results_ranked, y_val, d_target_mapping))

1735/1735 [==============================] - 2s 1ms/step


,n_acquisitions,is_prediction_in_top_3,capture_rate_top_3,product
0,3420,1574,46.02,sing_acquisition
1,9983,9418,94.34,shs_acquisition
2,14083,13191,93.67,tos_acquisition
3,4508,2946,65.35,wifi_acquisition
4,15586,13640,87.51,ttv_acquisition
5,844,423,50.12,sws_acquisition
6,5615,4816,85.77,hsic_acquisition
7,658,365,55.47,lwc_acquisition
8,36,0,0.00,hpro_acquisition
9,772,650,84.20,whsia_acquisition


In [19]:
n= 3
probabilities =  model.predict(X_val_normalized)
results_ranked = np.argsort(-probabilities, axis=1)
display(extract_stats(n, results_ranked, y_val, d_target_mapping))

1735/1735 [==============================] - 2s 1ms/step


,n_acquisitions,is_prediction_in_top_3,capture_rate_top_3,product
0,3420,1574,46.02,sing_acquisition
1,9983,9418,94.34,shs_acquisition
2,14083,13191,93.67,tos_acquisition
3,4508,2946,65.35,wifi_acquisition
4,15586,13640,87.51,ttv_acquisition
5,844,423,50.12,sws_acquisition
6,5615,4816,85.77,hsic_acquisition
7,658,365,55.47,lwc_acquisition
8,36,0,0.00,hpro_acquisition
9,772,650,84.20,whsia_acquisition


#### Tunning

In [20]:
#!pip install tensorflow scikeras scikit-learn

In [21]:
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import make_scorer

In [22]:
def create_model(input_size, output_size, hidden_layer_dim, activation):
    
    model = Sequential()
    model.add(Dense(64, activation=activation, input_dim=input_size))
    model.add(Dense(hidden_layer_dim, activation=activation))
    model.add(Dense(output_size, activation='softmax'))
    
    return model

In [23]:
# Create KerasClassifier
model = KerasClassifier(
    create_model,
    input_size=input_size,
    output_size=output_size,
    hidden_layer_dim=100,
    activation='relu',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3)]
)

In [24]:
from sklearn.metrics import top_k_accuracy_score

In [ ]:
# Define the hyperparameters to tune
param_grid = {
    'hidden_layer_dim': [50, 100, 200, 300, 400, 500],
    'optimizer': ['adam', 'sgd'],
    'optimizer__learning_rate': [0.0001, 0.0005, 0.001],
    'activation': ['relu', 'sigmoid'],
    'batch_size': [128, 256, 512, 1024, 2048,4096]
}

# Perform grid search
grid_search = GridSearchCV(
    estimator=model, 
    param_grid=param_grid, 
    cv=5, 
    scoring=make_scorer(top_k_accuracy_score, k=3, response_method='predict_proba')
    #scoring=make_scorer(top_k_accuracy_score, k=3, labels=list(d_target_mapping.values()))
)
grid_search.fit(X_train_normalized, y_train)

253/253 [==============================] - 2s 5ms/step - loss: 2.0931 - sparse_top_k_categorical_accuracy: 0.6198


In [ ]:
print(grid_search.best_score_, grid_search.best_params_)

In [ ]:
# Get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
best_params

In [ ]:
n= 3
probabilities =  best_model.predict_proba(X_val_normalized)
results_ranked = np.argsort(-probabilities, axis=1)
display(extract_stats(n, results_ranked, y_val, d_target_mapping))

In [ ]:
# Train the best model
best_model.fit(X_train_normalized, y_train, epochs=1000, validation_data=(X_test_normalized, y_test))

In [ ]:
n= 3
probabilities =  best_model.predict_proba(X_val_normalized)
results_ranked = np.argsort(-probabilities, axis=1)
display(extract_stats(n, results_ranked, y_val, d_target_mapping))

In [ ]:
print('ok')